# 1.1  Create an RDF Graph representing the same road network and traveltimes

In [7]:
%%capture
!pip3 install rdflib

In [8]:
import rdflib
from rdflib import Graph
from rdflib.namespace import DC, RDF, FOAF, RDFS
from rdflib import URIRef, BNode, Literal

In [ ]:


g = Graph()
# Graph using N3 syntax
n3data = """\
@prefix : <http://www.snee.com/ns/Railnetwork#> .
:Inverness :hasConnection :Aberdeen ;
           :distance      :3 . 
:Aberdeen :hasConnection :Inverness ;
      :distance    :2.5 .
:Aberdeen :hasConnection :Leuchars ;
           :distance    :1.5 .
:Leuchars :hasConnection :Aberdeen ;
           :distance    :1 .
:Leuchars :hasConnection :Edinburgh ;
           :distance    :1.5 .
:Edinburgh :hasConnection :Leuchars ;
           :distance    :3 .
:Edinburgh :hasConnection :Glasgow  ;
           :distance    :1  .
:Glasgow :hasConnection  :Edinburgh  ;
           :distance    :1  .
:Edinburgh :hasConnection :York  ;
           :distance    :3.5  .
:York     :hasConnection :Edinburgh  ;
           :distance    :4  .
:Glasgow  :hasConnection :Carlisle   ;
           :distance    :1  .
:Carlisle :hasConnection :Glasgow  ;
           :distance    :1  .
:Carlisle :hasConnection :York  ;
           :distance    :2.5  .
:York     :hasConnection :Carlisle   ;
           :distance    :3.5  .
:Birmingham :hasConnection :London  ;
           :distance    :1.5  .
:London   :hasConnection :Birmingham ;
           :distance    :2.5 .
:Leuchars :hasConnection :StAndrews ;
           :distance      :0.2 . 
:StAndrews :hasConnection :Leuchars ;
      :distance    :0.2 .
:York :hasConnection :London ;
           :distance    :1.8 .
:London :hasConnection :York ;
           :distance    :2.0 .
:London	:hasConnection :Brussels ;
			     :distance	  :2.5 .
:Brussels :hasConnection :London ;
			    :distance	:3.5  .	
:Brussels :hasConnection :Cologne ;
           :distance    :2 .
:Cologne :hasConnection :Brussels ;
           :distance    :1.5 .
:Toronto :hasConnection :Winnipeg  ;
           :distance    :36  .
:Winnipeg :hasConnection  :Toronto  ;
           :distance    :35  .
:Winnipeg :hasConnection :Saskatoon  ;
           :distance    :12  .
:Saskatoon :hasConnection :Winnipeg  ;
           :distance    :5  .
:Saskatoon :hasConnection :Edmonton  ;
           :distance    :12  .
:Edmonton :hasConnection :Saskatoon  ;
           :distance    :17  .
:Edmonton :hasConnection :Jasper  ;
           :distance    :6  .
:Jasper :hasConnection :Edmonton  ;
           :distance    :5  .
:Jasper :hasConnection :Vancouver  ;
           :distance    :12  .
:Vancouver :hasConnection :Jasper  ;
           :distance    :13  ."""
g.parse(data=n3data, format="n3")
#print all triples
for s, p, o in g:
   print((s, p, o))

# 1.2 Implement a SPARQL query returning all cities which can be reached from London.

In [ ]:
result = g.query(
    """SELECT  ?object
WHERE {
  :London :hasConnection ?object .
}
""")
for row in result:
    print(row)